**Chapter 2 – End-to-end Machine Learning project**

*This notebook contains the solution to the exercise 3 in chapter 2*

# Exercise 3

**Exercise**: _Try adding a `SelectFromModel` transformer in the preparation pipeline to select only the most important attributes._

In [16]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tarfile
import urllib
import joblib
from scipy import stats
from pandas.plotting import scatter_matrix
from packaging import version
from zlib import crc32
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array, check_is_fitted
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from scipy.stats import randint, uniform, geom, expon
from scipy.stats import loguniform

#from handson_ml3.my_folder.chapter_2.downloading_the_data import download_housing_data, LOCAL_PATH, HOUSING_PATH
from downloading_the_data import download_housing_data, LOCAL_PATH, HOUSING_PATH

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

In [17]:
housing = download_housing_data()
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [18]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

strat_train_set, strat_test_set = train_test_split(
    housing, test_size=0.2, stratify=housing["income_cat"], random_state=42)

for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [20]:
class ClusterSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(self.n_clusters, n_init=10,
                              random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self  # always return self!

    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)
    
    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]

In [21]:
def column_ratio(X):
    return X[:, [0]] / X[:, [1]]

def ratio_name(function_transformer, feature_names_in):
    return ["ratio"]  # feature names out

def ratio_pipeline():
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        StandardScaler())

log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler())
cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore"))
cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1., random_state=42)
default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"),
                                     StandardScaler())
preprocessing = ColumnTransformer([
        ("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]),
        ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]),
        ("people_per_house", ratio_pipeline(), ["population", "households"]),
        ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population",
                               "households", "median_income"]),
        ("geo", cluster_simil, ["latitude", "longitude"]),
        ("cat", cat_pipeline, make_column_selector(dtype_include=object)),
    ],
    remainder=default_num_pipeline)  # one column remaining: housing_median_age

In [22]:
housing_prepared = preprocessing.fit_transform(housing)
housing_prepared.shape

(16512, 24)

In [23]:
preprocessing.get_feature_names_out()

array(['bedrooms__ratio', 'rooms_per_house__ratio',
       'people_per_house__ratio', 'log__total_bedrooms',
       'log__total_rooms', 'log__population', 'log__households',
       'log__median_income', 'geo__Cluster 0 similarity',
       'geo__Cluster 1 similarity', 'geo__Cluster 2 similarity',
       'geo__Cluster 3 similarity', 'geo__Cluster 4 similarity',
       'geo__Cluster 5 similarity', 'geo__Cluster 6 similarity',
       'geo__Cluster 7 similarity', 'geo__Cluster 8 similarity',
       'geo__Cluster 9 similarity', 'cat__ocean_proximity_<1H OCEAN',
       'cat__ocean_proximity_INLAND', 'cat__ocean_proximity_ISLAND',
       'cat__ocean_proximity_NEAR BAY', 'cat__ocean_proximity_NEAR OCEAN',
       'remainder__housing_median_age'], dtype=object)

Let's create a new pipeline that runs RandomizedSearchCV for the threshold so we can try to get a better result:

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from scipy.stats import uniform

# Define the parameter distributions including the threshold
param_distribs = {
    'selector__threshold': uniform(0, 0.05),
    'svr__C': loguniform(20, 200_000),
    'svr__gamma': expon(scale=1.0),
    'svr__kernel': ['linear', 'rbf']
}

# Create the pipeline with a placeholder threshold value
selector_pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('selector', SelectFromModel(RandomForestRegressor(random_state=42))),
    ('svr', SVR()),
])

# Define the random search
rnd_search = RandomizedSearchCV(selector_pipeline,
                                param_distributions=param_distribs,
                                n_iter=50, cv=3,
                                scoring='neg_root_mean_squared_error',
                                verbose=2,
                                random_state=42)

rnd_search.fit(housing.iloc[:5000], housing_labels.iloc[:5000])

# Get the best parameters
best_threshold = rnd_search.best_params_['selector__threshold']
best_svr_C = rnd_search.best_params_['svr__C']
best_svr_gamma = rnd_search.best_params_['svr__gamma']
best_svr_kernel = rnd_search.best_params_['svr__kernel']

# Update the pipeline with the best parameters
selector_pipeline.set_params(
    selector__threshold=best_threshold,
    svr__C=best_svr_C,
    svr__gamma=best_svr_gamma,
    svr__kernel=best_svr_kernel
)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END selector__threshold=0.018727005942368126, svr__C=127024.42021281396, svr__gamma=1.3167456935454493, svr__kernel=linear; total time=   5.9s
[CV] END selector__threshold=0.018727005942368126, svr__C=127024.42021281396, svr__gamma=1.3167456935454493, svr__kernel=linear; total time=   6.1s
[CV] END selector__threshold=0.018727005942368126, svr__C=127024.42021281396, svr__gamma=1.3167456935454493, svr__kernel=linear; total time=   6.4s
[CV] END selector__threshold=0.02984250789732435, svr__C=1214.3978986882596, svr__gamma=0.10533264473289401, svr__kernel=linear; total time=   5.6s
[CV] END selector__threshold=0.02984250789732435, svr__C=1214.3978986882596, svr__gamma=0.10533264473289401, svr__kernel=linear; total time=   5.7s
[CV] END selector__threshold=0.02984250789732435, svr__C=1214.3978986882596, svr__gamma=0.10533264473289401, svr__kernel=linear; total time=   5.6s
[CV] END selector__threshold=0.04330880728874676, 

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardscaler',
                                                              StandardScaler())]),
                                   transformers=[('bedrooms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out=<function ratio_name at 0x7af9f5a...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7af9f5af0580>)])),
                ('selector',
                 SelectFromModel(estimator=RandomForestRegressor(random_state=42),
                                 threshold=0.007404346497669995)),
                ('svr', SVR(C=195880.83012927117, gamma=0.3103108690191677))])

In [27]:
best_threshold

0.007404346497669995

In [28]:
best_svr_C

195880.83012927117

In [29]:
best_svr_gamma

0.3103108690191677

In [30]:
best_svr_kernel

'rbf'

In [25]:
selector_rmses = -cross_val_score(selector_pipeline,
                                  housing.iloc[:5000],
                                  housing_labels.iloc[:5000],
                                  scoring="neg_root_mean_squared_error",
                                  cv=3)
pd.Series(selector_rmses).describe()

count        3.000000
mean     56713.692587
std       2011.579891
min      55241.366017
25%      55567.689915
50%      55894.013813
75%      57449.855872
max      59005.697931
dtype: float64

Well, it doesn't seem to have helped at all